In [1]:
from utils_filter import *
from utils_datetime import *
from dateutil import parser
from dateutil.relativedelta import relativedelta

Subsets outlooks, pph, and report data in various ways (add label to each datapoint for each method of subsetting so can be pulled out of full dataset later)


In [2]:
data_location = 'data'
moderate = False # only consider moderate days
outlooks, pph, reports = read_datasets(data_location, moderate)

reading outlooks 1
reading outlooks 2


c:\Users\miles\OneDrive\Documents\UW\Research\utils_filter.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outlooks = outlooks.append(gp.read_file(data_location + '/outlooks/' + mod_string + '_outlooks_2.shp'))


reading pph
reading storm reports


In [6]:
tz_conversions = {'PST': timedelta(hours=8),
                  'MST': timedelta(hours=7),
                  'CST': timedelta(hours=6),
                  'CSt': timedelta(hours=6),
                  'CSC': timedelta(hours=6),
                  'SCT': timedelta(hours=6),
                  'EST': timedelta(hours=5),
                  'ESt': timedelta(hours=5),
                  'PDT': timedelta(hours=7),
                  'MDT': timedelta(hours=6),
                  'CDT': timedelta(hours=5),
                  'EDT': timedelta(hours=4),
                  'HST': timedelta(hours=10),
                  'SST': timedelta(hours=11),
                  'GST': timedelta(hours=10),
                  'AKS': timedelta(hours=9),
                  'AST': timedelta(hours=4),
                  'UNK': timedelta(hours=5),
                  'GMT': timedelta(0)}

def get_reports_date_strings(date_times, timezones):
    # returns list of strings of date of given datetime and timezone (where day cutoffs are 12-12 UTC) formatted as 'YYYYMMDD0000'
    for datetime, timezone, i in zip(date_times, timezones, range(len(timezones))):
        #print(datetime + ' ' + timezone[:3])
        datetime = parser.parse(datetime)
        datetime = datetime + tz_conversions[timezone[:3]]
        #print(datetime)
        if (datetime.hour < 12):
            datetime = datetime - timedelta(days = 1)
        if datetime.year > 2049:
            datetime = datetime - relativedelta(years = 100)
        datetime = datetime.strftime("%Y%m%d") + '0000'
        if i == 0:
            ret = [datetime]
        else:
            ret.append(datetime)
    return ret

def get_pph_date_strings(times):
    # returns a list of strings of given dates formatted as 'YYYYMMDD0000'
    for datetime, i in zip(times, range(len(times))):
        string = datetime.dt.strftime("%Y%m%d").values + '0000'
        if i == 0:
            ret = [string]
        else:
            ret.append(string)
    return ret

In [4]:
# add dates to reports and pph in same format as in outlooks
reports['DATE'] = get_reports_date_strings(reports['BEGIN_DATE_TIME'], reports['CZ_TIMEZONE']) #TODO: or end time?
pph['time'] = get_pph_date_strings(pph.time) 

In [65]:
# subset outlooks into only one day 1, two day 2, and one day 3 categorical outlooks 
# day 3: cycle not -1. day 2: cycle not -1. Day 1: cycle 6. Category: categorical or none.
outlooks = outlooks[(((outlooks['DAY'] == 1) & (outlooks['CYCLE'] == 6)) | ((outlooks['DAY'] == 2) & (outlooks['CYCLE'] != -1)) | ((outlooks['DAY'] == 3) & (outlooks['CYCLE'] != -1)))
         & (outlooks['CATEGORY'] != 'WIND') & (outlooks['CATEGORY'] != 'HAIL') & (outlooks['CATEGORY'] != 'TORNADO')& (outlooks['CATEGORY'] != 'ANY SEVERE')& (outlooks['CATEGORY'] != 'PROB')]


In [67]:
def add_outlooks_label(outlooks, label_dates, labels, label_name, none_label):
    # adds new column with values from labels on the corresponding list of dates in label_dates (DONE)
    print("adding a new column in outlooks")
    outlooks[label_name] = none_label
    for label, dates in zip(labels, label_dates):
        #print(label)
        outlooks[label_name].loc[outlooks['DATE'].isin(dates)] = label
    return outlooks

def add_pph_label(pph, label_dates, labels, label_name, none_label):
    # adds new variable with values from labels on the corresponding list of dates in label_dates (DONE)
    print("adding new variable in pph")
    pph[label_name] = (('time'), np.full(len(pph['time']), none_label))
    for label, dates in zip(labels, label_dates):
        #print(label)
        pph[label_name].loc[pph['time'].isin(dates)] = label  
    return pph

def add_reports_label(reports, label_dates, labels, label_name, none_label):
    # adds new column with values from labels on the corresponding list of dates in label_dates (DONE)
    reports[label_name] = none_label
    print("adding a new column in reports")
    for label, dates in zip(labels, label_dates):
       #print(label)
       reports[label_name].loc[reports['DATE'].isin(dates)] = label # TODO: syntax may be wrong
    return reports

def add_labels(outlooks, pph, reports, label_dates, labels, label_name, none_label):
    # adds labels, overwriting with later ones if a date has multiple labels
    return(add_outlooks_label(outlooks, label_dates, labels, label_name, none_label), 
           add_pph_label(pph, label_dates, labels, label_name, none_label),
           add_reports_label(reports, label_dates, labels, label_name, none_label))



Subset by threshold

In [68]:
# add max threshold forecasted for valid day to each datapoint

categories = ['TSTM', 'MRGL', 'SLGT', 'ENH', 'MDT', 'HIGH']
category_dates = []
for category in categories:
    category_dates.append(identify_dates_above_threshold(outlooks, category))

(new_outlooks, new_pph, new_reports) = add_labels(outlooks, pph, reports, category_dates, categories, 'MAX_CAT', 'NONE')



adding a new column in outlooks
adding new variable in pph


C:\Users\miles\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\miles\AppData\Local\Temp\ipykernel_12212\2717672329.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outlooks[label_name].loc[outlooks['DATE'].isin(dates)] = label


adding a new column in reports


C:\Users\miles\AppData\Local\Temp\ipykernel_12212\2717672329.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reports[label_name].loc[reports['DATE'].isin(dates)] = label # TODO: syntax may be wrong


Subset by ramp up/down

In [ ]:
# define and add ramp category for each datapoint. Potentially add 2 binary ramp up and ramp down (4 options are [up, down, up and down, niether]). How many forecasts to consider for each day? The day 3, both day 2, and the first day 1 (so 4 forecasts ramp)

Subset by accurate/inaccurate

Consider forecast issue time for day 1--subset reports (and revise pph?)--look at other studies to see how they handled multiple day 1 outlooks (see the one in slack). Starting point should be use first forecasts, later ones would be a different analysis for another time. Decide if we should denote these somehow..

Subset by season

In [71]:
# add column denoting season (4 met seasons as starting point)

'198701010000'

Subset by region

In [ ]:
# decide how to do... lower priotity, could do by state of center of highest category (or center of PPH?) Could start with super basic version

Subset by environmental data (to do later)

In [74]:
# resave labelled data
outlook_save_location = 'data/outlooks'
pph_save_location = 'data/pph'
report_save_location = 'data/storm_reports'

new_outlooks.to_file(outlook_save_location + '/labelled_outlooks.shp')
new_pph.to_netcdf(pph_save_location + '/labelled_pph.nc')
new_reports.to_csv(report_save_location + '/labelled_reports.csv')

C:\Users\miles\AppData\Local\Temp\ipykernel_12212\2456175457.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  new_outlooks.to_file(outlook_save_location + '/labelled_outlooks.shp')


In [75]:
# TODO: create master function that subsets data based on easy inputs once it's all been labelled. Put in a util file
new_outlooks

,ISSUE,EXPIRE,PRODISS,TYPE,DAY,THRESHOLD,CATEGORY,CYCLE,DATE,geometry,Categorical Risk
36,198701091200,198701101200,198701080719,C,2,None,None,7,198701090000,None,NONE
38,198701091200,198701101200,198701081731,C,2,None,None,17,198701090000,None,NONE
73,198701161200,198701171200,198701160615,C,1,TSTM,CATEGORICAL,6,198701160000,"POLYGON ((-81.10300 30.60200, -81.08700 30.521...",TSTM
85,198701191200,198701201200,198701180703,C,2,None,None,7,198701190000,None,NONE
104,198701221200,198701231200,198701210800,C,2,None,None,7,198701220000,None,NONE
...,...,...,...,...,...,...,...,...,...,...,...
183175,202312311200,202401011200,202312301607,C,2,None,None,17,202312310000,None,TSTM
183178,202312311200,202401011200,202312310520,C,1,None,None,6,202312310000,None,TSTM
183179,202401011200,202401021200,202312310600,C,2,None,None,7,202401010000,None,NONE
183180,202401021200,202401031200,202312310815,C,3,TSTM,CATEGORICAL,8,202401020000,"POLYGON ((-90.74500 28.71100, -91.08300 28.732...",TSTM
